In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import sys
sys.path.append('../torch')
from conv_emergence import NeuralNet, compute_entropy

def plot_receptive_field(weights, figsize=(10, 5)):
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    weights = weights[np.argsort(compute_entropy(weights))]
    ax.imshow(weights, cmap='gray')#, vmin=-1, vmax=1)
    ax.set_xlabel('Input neurons')
    ax.set_ylabel('Hidden neurons')
    return fig, ax

def plot_metrics(metrics, figsize=(10, 5)):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=figsize)
    ax1.plot(metrics['losses'])
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    
    ax2.plot(metrics['accs'])
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    
    # num_epochs = len(metrics['losses'])
    # every_epoch = min(max(num_epochs // 100, 1), 500)
    # ipr_epochs = np.append(np.arange(0, num_epochs, every_epoch), num_epochs - 1)
    ax3.plot(metrics['iprs'])
    ax3.set_xlabel('Epoch intervals')
    ax3.set_ylabel('IPR')
    
    return fig, (ax1, ax2, ax3)

def load_and_viz(xi1, xi2, gain, L, K, dim, batch_size, num_epochs, loss, lr, activation, second_layer, figsize1=(10, 4), figsize2=(12, 4)):
    key = f'__xi1={xi1:05.2f}_xi2={xi2:05.2f}_gain={gain:05.2f}_L={L:03d}_K={K:03d}_dim={dim}_batch_size={batch_size}_num_epochs={num_epochs}_loss={loss}_lr={lr:.3f}_activation={activation}_second_layer={second_layer}'
    model = NeuralNet(L, K, activation, second_layer)
    model.load_state_dict(torch.load(f'./results/weights_{key}.pt'))
    weights = model.ff1.weight.detach().numpy()
    
    title = f'xi1={xi1:.2f}, xi2={xi2:.2f}, gain={gain:.2f}, L={L}, K={K}, dim={dim}, batch_size={batch_size}, num_epochs={num_epochs}, loss={loss}, lr={lr:.3f}, activation={activation}, second_layer={second_layer}'
    fig, ax = plot_receptive_field(weights, figsize=figsize1)
    fig.suptitle(title)
    fig.savefig(f'./figs/receptive_field_{key}.png', dpi=300)
    # plt.show(fig)
    plt.close(fig)
    
    metrics = np.load(f'./results/metrics_{key}.npz', allow_pickle=True)
    fig, (ax1, ax2, ax3) = plot_metrics(metrics, figsize=figsize2)
    fig.suptitle(title)
    fig.savefig(f'./figs/metrics_{key}.png', dpi=300)
    # plt.show(fig)
    plt.close(fig)
    
    return weights, metrics

In [2]:
# define parameter range  
xi1 = (np.sqrt(20), 10.)
xi2 = (0.1, np.sqrt(10),)
gain = (0.1, 1.1, 3.0,)
L = (20, 40, 100,)
K = (10, 40, 100,)
dim = (1,)
batch_size = 1000
num_epochs = 1000000
lr = 0.1
second_layer = ('linear', 'learnable_bias', 0.,)
path = ('/ceph/scratch/leonl/results',)  
activation = 'tanh'

### Metrics

In [3]:
xi1 = (10., np.sqrt(20),)#[1]
xi2 = (np.sqrt(10), 0.1,)#[1]
gain = (0.1, 1.1, 3.0,)#[0]
L = (20, 40, 100,)#[0]
K = (10, 40, 100,)#[2]
second_layer = ('linear', 'learnable_bias', 0.,)#[2]
from itertools import product
for xi1_, xi2_, gain_, L_, K_, second_layer_ in product(xi1, xi2, gain, L, K, second_layer):
    try:
        weights, metrics = load_and_viz(xi1_, xi2_, gain_, L_, K_, 1, batch_size, num_epochs, 'mse', lr, activation, second_layer_)
    except:
        print(f'Failed: xi1={xi1_}, xi2={xi2_}, gain={gain_}, L={L_}, K={K_}, second_layer={second_layer_}')
# weights, metrics = load_and_viz(xi1, xi2, gain, L, K, 1, batch_size, num_epochs, 'mse', lr, activation, second_layer)

/Users/leonlufkin/Documents/GitHub/Localization/leonl/../torch/conv_emergence.py:51: RuntimeWarning: divide by zero encountered in divide
  prob = count / np.sum(count)
/Users/leonlufkin/Documents/GitHub/Localization/leonl/../torch/conv_emergence.py:51: RuntimeWarning: invalid value encountered in divide
  prob = count / np.sum(count)


Failed: xi1=10.0, xi2=3.1622776601683795, gain=0.1, L=100, K=10, second_layer=0.0
Failed: xi1=10.0, xi2=3.1622776601683795, gain=0.1, L=100, K=40, second_layer=0.0
Failed: xi1=10.0, xi2=3.1622776601683795, gain=0.1, L=100, K=100, second_layer=0.0
Failed: xi1=10.0, xi2=3.1622776601683795, gain=1.1, L=100, K=10, second_layer=learnable_bias
Failed: xi1=10.0, xi2=3.1622776601683795, gain=1.1, L=100, K=10, second_layer=0.0
Failed: xi1=10.0, xi2=3.1622776601683795, gain=1.1, L=100, K=100, second_layer=0.0
Failed: xi1=10.0, xi2=3.1622776601683795, gain=3.0, L=100, K=10, second_layer=linear
Failed: xi1=10.0, xi2=3.1622776601683795, gain=3.0, L=100, K=10, second_layer=learnable_bias
Failed: xi1=10.0, xi2=3.1622776601683795, gain=3.0, L=100, K=10, second_layer=0.0
Failed: xi1=10.0, xi2=3.1622776601683795, gain=3.0, L=100, K=40, second_layer=0.0
Failed: xi1=10.0, xi2=0.1, gain=0.1, L=100, K=10, second_layer=0.0
Failed: xi1=10.0, xi2=0.1, gain=1.1, L=100, K=10, second_layer=0.0
Failed: xi1=10.0, x